# Stage 4: Clustering

**Primary author:** Victoria
**Builds on:**
- *Hierarchical_Clustering_Indicators_with_BGE_M3_Embeddings.ipynb* (Victoria — HDBSCAN; Sahana — agglomerative/Ward's)
- *NC_Indicators_wo_Context_Clustered_Embeddings_Compared.ipynb* (Nathan — DBSCAN, EmbeddingComparison framework, silhouette evaluation)
- `03_dimensionality_reduction.ipynb` (Stage 3 output: UMAP embeddings)

**Prompt engineering:** Victoria
**AI assistance:** Claude (Anthropic)
**Environment:** Great Lakes for parameter sweeps, Colab for single runs

This notebook applies unsupervised clustering to the 10-dimensional UMAP embeddings
of 12,622 verified CCC indicators. Two clustering approaches are used:

1. **HDBSCAN** — a density-based method that discovers clusters of varying density
   and automatically determines the number of clusters. Requires an epsilon
   sensitivity analysis (per KCT, Feb 15).
2. **Agglomerative clustering with Ward's method** — a hierarchical method that
   merges clusters bottom-up using global variance statistics. Requires a
   predetermined number of clusters (k), which we set to theory-motivated values
   derived from expert CCC references.

The notebook follows a principled workflow:
- Analyze the pairwise distance distribution to understand the data's distance scale
- Use that distribution to select epsilon candidates for HDBSCAN
- Run a sensitivity analysis across epsilon values
- Compare HDBSCAN results to agglomerative clustering at several granularities
- Visualize and qualitatively inspect the best results

## Running on Google Colab

If running on Google Colab:

1. Go to **Runtime > Change runtime type**
2. A GPU is not required for clustering, but will speed up UMAP if you need to
   regenerate embeddings
3. Click **Save**, then run all cells

For parameter sweeps (many HDBSCAN runs), Great Lakes is recommended.
Single clustering runs complete in seconds on any hardware.

---
## Section 1: Setup and Data Preparation

### Imports

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
)
from scipy.spatial.distance import pdist

import hdbscan  # install via: pip install hdbscan

import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['figure.dpi'] = 120

np.random.seed(42)

### Environment Auto-Detection and Paths

In [ ]:
# --- Environment Auto-Detection ---
try:
    IS_COLAB = 'google.colab' in str(get_ipython())
except NameError:
    IS_COLAB = False

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_ROOT = Path('/content/drive/MyDrive/SIADS 692 Milestone II/Milestone II - NLP Cryptic Crossword Clues')
else:
    try:
        PROJECT_ROOT = Path(__file__).resolve().parent.parent
    except NameError:
        PROJECT_ROOT = Path.cwd().parent

DATA_DIR = PROJECT_ROOT / 'data'
OUTPUT_DIR = PROJECT_ROOT / 'outputs'
FIGURES_DIR = OUTPUT_DIR / 'figures'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

print(f'Project root: {PROJECT_ROOT}')
print(f'Data directory: {DATA_DIR}')
print(f'Output directory: {OUTPUT_DIR}')
print(f'Figures directory: {FIGURES_DIR}')

### Input File Validation

Before proceeding, we verify that all required input files from earlier stages exist.
These are:

| File | Produced by | Description |
|------|-------------|-------------|
| `embeddings_umap_10d.npy` | Stage 3 | 10D UMAP embeddings for clustering |
| `embeddings_umap_2d.npy` | Stage 3 | 2D UMAP embeddings for visualization |
| `indicator_index_all.csv` | Stage 2 | Row-to-indicator-string mapping |
| `verified_clues_labeled.csv` | Stage 1 | Clue-indicator pairs with wordplay labels |

In [ ]:
required_files = {
    'embeddings_umap_10d.npy': 'Run 03_dimensionality_reduction.ipynb (Stage 3)',
    'embeddings_umap_2d.npy': 'Run 03_dimensionality_reduction.ipynb (Stage 3)',
    'indicator_index_all.csv': 'Run 02_embedding_generation.ipynb (Stage 2)',
    'verified_clues_labeled.csv': 'Run 01_data_cleaning.ipynb (Stage 1)',
}

for fname, fix_msg in required_files.items():
    fpath = DATA_DIR / fname
    assert fpath.exists(), (
        f'Missing required file: {fpath}\n'
        f'Fix: {fix_msg}'
    )

print('All input files found:')
for fname in required_files:
    print(f'  {fname}')

### Load Input Files

In [ ]:
# Load UMAP embeddings
embeddings_10d = np.load(DATA_DIR / 'embeddings_umap_10d.npy')
embeddings_2d = np.load(DATA_DIR / 'embeddings_umap_2d.npy')

# Load indicator index (maps row i -> indicator string)
df_index = pd.read_csv(DATA_DIR / 'indicator_index_all.csv', index_col=0)

# Load clue-level labels
df_labels = pd.read_csv(DATA_DIR / 'verified_clues_labeled.csv')

print(f'10D embeddings shape: {embeddings_10d.shape}')
print(f'2D embeddings shape:  {embeddings_2d.shape}')
print(f'Indicator index rows: {len(df_index)}')
print(f'Clue-label rows:      {len(df_labels)}')

# Sanity checks
n_indicators = len(df_index)
assert embeddings_10d.shape == (n_indicators, 10), (
    f'Expected 10D shape ({n_indicators}, 10), got {embeddings_10d.shape}'
)
assert embeddings_2d.shape == (n_indicators, 2), (
    f'Expected 2D shape ({n_indicators}, 2), got {embeddings_2d.shape}'
)
print(f'\nShape checks passed: {n_indicators:,} indicators')

### Build Per-Indicator Label Sets

The label file `verified_clues_labeled.csv` contains one row per (clue_id, indicator)
pair — 76,015 rows total. Each row has two independent label columns:

- **`wordplay_ho`** — the wordplay type from George Ho's blog parsing. Covers all 8
  types (anagram, reversal, hidden, container, insertion, deletion, homophone,
  alternation). Available for every row.
- **`wordplay_gt`** — algorithmically derived ground truth. Checks whether the answer
  can be produced by applying the wordplay operation to the clue text (e.g., is the
  answer a permutation of the fodder letters? Is the answer hidden consecutively in
  the clue?). Covers only 4 types (hidden, reversal, alternation, anagram) and is
  null for many rows where no pattern fires or the answer is too short.

**These are two independent classification systems and should not be merged.** Ho labels
come from human blog commentary; GT labels come from algorithmic pattern matching. They
agree 92.6% of the time where GT exists.

**Multi-label indicators:** A single indicator string can appear under multiple wordplay
types. For example, "about" appears as container, reversal, and anagram under Ho labels.
This is a genuine linguistic property of CCC indicators — the same word can signal
different operations in different clue contexts. We preserve this by collecting the
*set* of distinct labels per unique indicator, rather than collapsing to a single
"primary" label.

In [ ]:
# Get the indicator column name from the index file
indicator_col = df_index.columns[0]  # should be 'indicator'

# Group the clue-level labels by indicator to get the set of Ho labels
# and the set of GT labels for each unique indicator
ho_labels_per_indicator = (
    df_labels
    .dropna(subset=['wordplay_ho'])
    .groupby('indicator')['wordplay_ho']
    .apply(lambda x: set(x.unique()))
    .rename('ho_label_set')
)

gt_labels_per_indicator = (
    df_labels
    .dropna(subset=['wordplay_gt'])
    .groupby('indicator')['wordplay_gt']
    .apply(lambda x: set(x.unique()))
    .rename('gt_label_set')
)

# Join onto the indicator index
df_indicators = df_index.copy()
df_indicators = df_indicators.merge(
    ho_labels_per_indicator, left_on=indicator_col, right_index=True, how='left'
)
df_indicators = df_indicators.merge(
    gt_labels_per_indicator, left_on=indicator_col, right_index=True, how='left'
)

# For indicators with no Ho labels in the labeled file, fill with empty set
df_indicators['ho_label_set'] = df_indicators['ho_label_set'].apply(
    lambda x: x if isinstance(x, set) else set()
)
df_indicators['gt_label_set'] = df_indicators['gt_label_set'].apply(
    lambda x: x if isinstance(x, set) else set()
)

# Count how many Ho labels each indicator has
df_indicators['n_ho_labels'] = df_indicators['ho_label_set'].apply(len)

print(f'Indicators with Ho labels: {(df_indicators["n_ho_labels"] > 0).sum():,}')
print(f'Indicators with no Ho labels: {(df_indicators["n_ho_labels"] == 0).sum():,}')
print(f'\nMulti-label distribution:')
print(df_indicators['n_ho_labels'].value_counts().sort_index().to_string())

For visualization purposes, we also create a "primary Ho label" column that picks the
single most common Ho label for each indicator. This is used **only** for coloring
scatter plots where each point needs a single color. The full label set
(`ho_label_set`) is used for all quantitative evaluation.

In [ ]:
# For scatter plot coloring: pick the most frequent Ho label per indicator.
# When an indicator has multiple Ho labels (e.g., "about" = container, reversal,
# anagram), we pick the one that appears in the most clues for that indicator.
primary_ho = (
    df_labels
    .dropna(subset=['wordplay_ho'])
    .groupby(['indicator', 'wordplay_ho'])
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
    .drop_duplicates(subset='indicator', keep='first')
    .set_index('indicator')['wordplay_ho']
    .rename('primary_ho_label')
)

df_indicators = df_indicators.merge(
    primary_ho, left_on=indicator_col, right_index=True, how='left'
)

print('Primary Ho label distribution:')
print(df_indicators['primary_ho_label'].value_counts().to_string())

---
## Section 2: Pairwise Distance Analysis

**Why this step is necessary:** HDBSCAN's `cluster_selection_epsilon` parameter defines
a distance threshold below which clusters will be merged. If we choose an epsilon of 0.5
but all pairwise distances in our data are between 0.01 and 0.05, that epsilon is
meaninglessly large — it would merge everything into one cluster. Conversely, an epsilon
of 0.001 in that same data would have no effect. **Without knowing the actual distance
scale, any epsilon choice is arbitrary.**

Following KCT's guidance (Feb 15 meeting): "Take a look at typical pairwise distances,
get a distribution, pick a value of epsilon based on that."

We compute pairwise Euclidean distances on a random sample of 2,000 points from the
10D UMAP embeddings. Using all 12,622 points would produce ~80 million distance pairs
(12,622 × 12,621 / 2), which is computationally expensive. A sample of 2,000 gives
~2 million pairs — more than enough to characterize the distribution.

**Note on distance metric:** HDBSCAN uses Euclidean distance by default, and our 10D
UMAP embeddings were produced with `metric='cosine'` — meaning UMAP already arranged
the points so that cosine-similar items are Euclidean-close in the reduced space. So
Euclidean distance in UMAP space is appropriate here.

In [ ]:
# Sample 2,000 points for pairwise distance computation
sample_size = 2000
rng = np.random.RandomState(42)
sample_idx = rng.choice(len(embeddings_10d), size=sample_size, replace=False)
sample_embeddings = embeddings_10d[sample_idx]

# Compute pairwise Euclidean distances (returns condensed form)
# pdist returns a 1D array of all unique pairs
pairwise_dists = pdist(sample_embeddings, metric='euclidean')

print(f'Sample size: {sample_size}')
print(f'Number of pairwise distances: {len(pairwise_dists):,}')
print(f'\nDistance statistics:')
print(f'  Min:    {pairwise_dists.min():.4f}')
print(f'  Max:    {pairwise_dists.max():.4f}')
print(f'  Mean:   {pairwise_dists.mean():.4f}')
print(f'  Median: {np.median(pairwise_dists):.4f}')
print(f'  Std:    {pairwise_dists.std():.4f}')

# Key percentiles — these will guide epsilon selection
percentiles = [5, 10, 25, 50, 75, 90, 95]
percentile_values = np.percentile(pairwise_dists, percentiles)

print(f'\nKey percentiles:')
for p, v in zip(percentiles, percentile_values):
    print(f'  {p:3d}th percentile: {v:.4f}')

### Distance Distribution Histogram

The histogram below shows the distribution of all pairwise Euclidean distances in the
2,000-point sample. Vertical lines mark key percentiles. The shape of this distribution
tells us:

- **If unimodal and tight:** Most points are at similar distances — the data is
  relatively uniform, and clustering will be sensitive to epsilon.
- **If multimodal or with a long tail:** There are distinct distance scales — some
  points are much closer to each other than to the bulk, suggesting natural clusters.
- **Where the bulk of the distribution sits:** This sets the scale for epsilon selection.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax.hist(pairwise_dists, bins=200, color='steelblue', alpha=0.7, edgecolor='none',
        density=True)

# Mark key percentiles
colors_pctl = ['#e41a1c', '#ff7f00', '#4daf4a', '#377eb8', '#4daf4a', '#ff7f00', '#e41a1c']
for p, v, c in zip(percentiles, percentile_values, colors_pctl):
    ax.axvline(v, color=c, linestyle='--', alpha=0.7, linewidth=1.2,
               label=f'{p}th pctl = {v:.3f}')

ax.set_xlabel('Euclidean Distance (10D UMAP space)')
ax.set_ylabel('Density')
ax.set_title('Pairwise Distance Distribution (2,000-point sample)')
ax.legend(fontsize=8, loc='upper right')

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'pairwise_distance_distribution.png', dpi=150,
            bbox_inches='tight')
plt.show()

print(f'Saved: {FIGURES_DIR / "pairwise_distance_distribution.png"}')

---
## Section 3: HDBSCAN with Epsilon Sensitivity Analysis

### What is HDBSCAN?

**HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise)**
is a density-based clustering algorithm. Unlike k-means, it does not require the number
of clusters to be specified in advance — it discovers clusters of varying density
automatically. Points in low-density regions are labeled as "noise" (cluster label = -1)
rather than being forced into the nearest cluster.

### Key Parameters

- **`min_cluster_size`** — The minimum number of points required to form a cluster.
  We use 10, meaning any group of fewer than 10 nearby indicators is treated as noise
  rather than a cluster. This prevents tiny, meaningless clusters.

- **`cluster_selection_epsilon`** — A distance threshold for cluster merging. When two
  clusters are closer than epsilon, HDBSCAN merges them into one. Larger epsilon values
  produce fewer, larger clusters; smaller values produce more, finer-grained clusters.
  **This is the parameter we will sweep** across values informed by the distance
  distribution in Section 2.

### Interpreting the Sensitivity Analysis

We run HDBSCAN at several epsilon values and track:
- **Number of clusters found** — How granularity changes with epsilon
- **Noise percentage** — How many points are left unassigned
- **Silhouette score** — Measures how similar each point is to its own cluster vs.
  the nearest other cluster (-1 to 1; higher is better). Computed on non-noise
  points only.
- **Davies-Bouldin index** — Measures the average similarity between each cluster
  and its most similar cluster (lower is better).

**Stability interpretation:** If metrics change dramatically with small epsilon changes,
the cluster structure is **fragile** — it depends sensitively on a parameter choice
rather than reflecting genuine structure. If metrics are stable across a range of epsilon
values, the structure is **robust** and we can be more confident in the results.

### Select Epsilon Candidates

We choose epsilon candidates based on the distance distribution percentiles computed
above. The candidates span from a small value (which will produce many fine-grained
clusters) to a larger value (which will merge clusters aggressively). We also include
0.0 as a baseline (no merging — pure HDBSCAN without epsilon smoothing).

In [ ]:
# Select epsilon candidates from the distance distribution
# We use percentiles of the pairwise distance distribution as a principled basis.
# The candidates range from 0.0 (no epsilon merging) to around the 25th percentile.
# Values beyond the median would merge nearly everything.
epsilon_candidates = [
    0.0,                                            # baseline: no epsilon merging
    float(np.percentile(pairwise_dists, 1)),        # ~1st percentile
    float(np.percentile(pairwise_dists, 5)),        # ~5th percentile
    float(np.percentile(pairwise_dists, 10)),       # ~10th percentile
    float(np.percentile(pairwise_dists, 15)),       # ~15th percentile
    float(np.percentile(pairwise_dists, 20)),       # ~20th percentile
    float(np.percentile(pairwise_dists, 25)),       # ~25th percentile
]

# Round for cleaner display
epsilon_candidates = [round(e, 4) for e in epsilon_candidates]

print('Epsilon candidates for HDBSCAN sweep:')
for i, eps in enumerate(epsilon_candidates):
    print(f'  {i+1}. epsilon = {eps}')

### Run HDBSCAN Sweep

For each epsilon value, we run HDBSCAN on the 10D UMAP embeddings and record
clustering metrics. Silhouette and Davies-Bouldin scores are computed on non-noise
points only (since noise points have no cluster assignment). This is standard practice
but means that a run with very high noise (few clustered points) can have a deceptively
high silhouette score — we must examine noise percentage alongside silhouette.

In [ ]:
hdbscan_results = []
hdbscan_labels_dict = {}  # store labels for each epsilon

for eps in epsilon_candidates:
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=10,
        cluster_selection_epsilon=eps,
        metric='euclidean',
        core_dist_n_jobs=-1,
    )
    labels = clusterer.fit_predict(embeddings_10d)
    hdbscan_labels_dict[eps] = labels

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = (labels == -1).sum()
    noise_pct = n_noise / len(labels) * 100

    # Compute metrics on non-noise points only
    non_noise_mask = labels != -1
    n_clustered = non_noise_mask.sum()

    if n_clusters >= 2 and n_clustered >= 2:
        sil = silhouette_score(embeddings_10d[non_noise_mask], labels[non_noise_mask])
        db = davies_bouldin_score(embeddings_10d[non_noise_mask], labels[non_noise_mask])
    else:
        sil = float('nan')
        db = float('nan')

    hdbscan_results.append({
        'epsilon': eps,
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'noise_pct': noise_pct,
        'n_clustered': n_clustered,
        'silhouette': sil,
        'davies_bouldin': db,
    })

    print(f'eps={eps:.4f}: {n_clusters:4d} clusters, '
          f'{n_noise:5d} noise ({noise_pct:5.1f}%), '
          f'silhouette={sil:.3f}, DB={db:.3f}')

df_hdbscan = pd.DataFrame(hdbscan_results)
print('\n--- HDBSCAN Sweep Summary ---')
print(df_hdbscan.to_string(index=False))

### Sensitivity Plot

The plot below shows how three key metrics change as epsilon increases. This is the
sensitivity analysis KCT requested. We look for:

- **Stable regions:** Where metrics plateau, indicating robust structure
- **Sharp transitions:** Where a small epsilon change causes large metric jumps,
  indicating fragile structure at that scale
- **Trade-offs:** More clusters typically means more noise points but potentially
  higher silhouette (tighter individual clusters)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Panel 1: Number of clusters
axes[0].plot(df_hdbscan['epsilon'], df_hdbscan['n_clusters'],
             'o-', color='steelblue', linewidth=2, markersize=6)
axes[0].set_xlabel('Epsilon')
axes[0].set_ylabel('Number of Clusters')
axes[0].set_title('Clusters Found')
axes[0].grid(True, alpha=0.3)

# Panel 2: Noise percentage
axes[1].plot(df_hdbscan['epsilon'], df_hdbscan['noise_pct'],
             'o-', color='#e41a1c', linewidth=2, markersize=6)
axes[1].set_xlabel('Epsilon')
axes[1].set_ylabel('Noise Points (%)')
axes[1].set_title('Noise Percentage')
axes[1].grid(True, alpha=0.3)

# Panel 3: Silhouette score
axes[2].plot(df_hdbscan['epsilon'], df_hdbscan['silhouette'],
             'o-', color='#4daf4a', linewidth=2, markersize=6)
axes[2].set_xlabel('Epsilon')
axes[2].set_ylabel('Silhouette Score')
axes[2].set_title('Silhouette Score (non-noise only)')
axes[2].grid(True, alpha=0.3)

plt.suptitle('HDBSCAN Epsilon Sensitivity Analysis', fontsize=14, y=1.02)
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'hdbscan_epsilon_sensitivity.png', dpi=150,
            bbox_inches='tight')
plt.show()

print(f'Saved: {FIGURES_DIR / "hdbscan_epsilon_sensitivity.png"}')

### Identify Best HDBSCAN Run

We select the HDBSCAN run with the highest silhouette score as the "best" run for
visualization and qualitative inspection later in this notebook. This is a pragmatic
choice — silhouette score is not the only criterion, and we save all runs so that
Notebook 05 can compare them more thoroughly.

In [ ]:
best_hdbscan_idx = df_hdbscan['silhouette'].idxmax()
best_hdbscan_row = df_hdbscan.loc[best_hdbscan_idx]
best_eps = best_hdbscan_row['epsilon']
best_hdbscan_labels = hdbscan_labels_dict[best_eps]

print(f'Best HDBSCAN run by silhouette score:')
print(f'  Epsilon:    {best_eps}')
print(f'  Clusters:   {int(best_hdbscan_row["n_clusters"])}')
print(f'  Noise:      {int(best_hdbscan_row["n_noise"])} ({best_hdbscan_row["noise_pct"]:.1f}%)')
print(f'  Silhouette: {best_hdbscan_row["silhouette"]:.3f}')
print(f'  Davies-Bouldin: {best_hdbscan_row["davies_bouldin"]:.3f}')

---
## Section 4: Agglomerative Clustering with Ward's Method

### Why Ward's Method?

**Agglomerative clustering** is a bottom-up hierarchical method: each point starts as
its own cluster, and pairs of clusters are merged iteratively until the desired number
of clusters (k) is reached.

**Ward's linkage** minimizes the total within-cluster variance at each merge step. As
KCT recommended (Feb 15 meeting): "Use Ward's method. It uses more global statistics
about the cluster, so doesn't form strange elongated clusters." Unlike single-linkage
(which can chain together distant points through intermediate neighbors) or average-
linkage (which can produce uneven cluster sizes), Ward's method produces compact,
roughly spherical clusters.

### Theory-Motivated k Values

Unlike HDBSCAN, agglomerative clustering requires the number of clusters (k) to be
specified. We choose k values based on the theoretical framework for CCC indicator
structure, with each k corresponding to a specific level of the conceptual hierarchy.
The source for each k is the `wordplay_seeds.xlsx` spreadsheet, compiled from expert
CCC references:

| k | Source | Meaning |
|---|--------|---------|
| 6 | `cc_for_dummies_ho_6` | Cryptic Crosswords for Dummies mapped to 6 of Ho's types |
| 8 | Ho's labeled types | All 8 wordplay types in the dataset |
| 12 | `minute_cryptic_ho_7` | Minute Cryptic categories with subcategories totaling 12 groups |
| 26 | `minute_cryptic_ALL` | Minute Cryptic full subcategories (most granular expert view) |
| 34 | `conceptual_groups` | Victoria's two-tier conceptual categories (organized by metaphor) |

These are theory-motivated choices, not arbitrary. Each represents a different hypothesis
about the natural granularity of indicator language.

In [ ]:
k_values = [6, 8, 12, 26, 34]
k_descriptions = {
    6: 'CC for Dummies -> 6 Ho types',
    8: 'All 8 Ho wordplay types',
    12: 'Minute Cryptic -> Ho (12 subcategories)',
    26: 'Minute Cryptic ALL (26 subcategories)',
    34: 'Conceptual groups (34 categories)',
}

agglo_results = []
agglo_labels_dict = {}  # store labels for each k

for k in k_values:
    clusterer = AgglomerativeClustering(
        n_clusters=k,
        linkage='ward',
    )
    labels = clusterer.fit_predict(embeddings_10d)
    agglo_labels_dict[k] = labels

    sil = silhouette_score(embeddings_10d, labels)
    db = davies_bouldin_score(embeddings_10d, labels)
    ch = calinski_harabasz_score(embeddings_10d, labels)

    agglo_results.append({
        'k': k,
        'description': k_descriptions[k],
        'silhouette': sil,
        'davies_bouldin': db,
        'calinski_harabasz': ch,
    })

    print(f'k={k:2d} ({k_descriptions[k]:>40s}): '
          f'silhouette={sil:.3f}, DB={db:.3f}, CH={ch:.0f}')

df_agglo = pd.DataFrame(agglo_results)
print('\n--- Agglomerative Clustering Summary ---')
print(df_agglo.to_string(index=False))

### Interpreting the Metrics

- **Silhouette score** (-1 to 1): Higher means points are well-matched to their own
  cluster and poorly matched to neighboring clusters. Scores above 0.3 suggest
  reasonable structure; above 0.5 is strong.
- **Davies-Bouldin index** (0 to ∞): Lower is better. Measures the average ratio of
  within-cluster scatter to between-cluster separation. A score below 1.0 means
  clusters are more separated than they are scattered.
- **Calinski-Harabasz index** (0 to ∞): Higher is better. Ratio of between-cluster
  dispersion to within-cluster dispersion. Tends to favor larger k values, so
  interpret with caution.

Note that agglomerative clustering assigns every point to a cluster (no noise points),
which makes its silhouette scores directly comparable across k values but not directly
comparable to HDBSCAN (which excludes noise points from the calculation).

---
## Section 5: Cluster Visualization

We visualize two clustering runs in detail:
1. **Agglomerative k=8** — the baseline hypothesis that indicators cluster into the
   8 labeled wordplay types
2. **Best HDBSCAN** — the best density-based result by silhouette score

For each run, we produce:
- A scatter plot of the 2D UMAP projection colored by cluster assignment
- Eight scatter plots (one per Ho wordplay type) showing where each type's indicators
  landed across the clusters

### Why Per-Type Plots?

Instead of asking "what is the correct label for this cluster?" we ask **"where did all
the anagram indicators end up?"** This approach examines clustering from the label side
and handles multi-label indicators naturally — an indicator like "about" (which is
container, reversal, and anagram under Ho labels) simply appears highlighted in all
three of those type-specific plots.

### Visualization Helper Functions

In [ ]:
# The 8 Ho wordplay types
HO_TYPES = ['anagram', 'reversal', 'hidden', 'container', 'insertion',
             'deletion', 'homophone', 'alternation']

# Consistent color palette for the 8 Ho wordplay types
HO_COLORS = {
    'anagram': '#e41a1c',
    'reversal': '#377eb8',
    'hidden': '#4daf4a',
    'container': '#984ea3',
    'insertion': '#ff7f00',
    'deletion': '#a65628',
    'homophone': '#f781bf',
    'alternation': '#999999',
}


def plot_clusters(embeddings_2d, labels, title, filename, noise_label=-1):
    """Scatter plot of 2D UMAP colored by cluster assignment."""
    fig, ax = plt.subplots(figsize=(12, 9))

    # Plot noise points first (gray, small)
    noise_mask = labels == noise_label
    if noise_mask.any():
        ax.scatter(
            embeddings_2d[noise_mask, 0],
            embeddings_2d[noise_mask, 1],
            s=1, alpha=0.15, color='lightgray', label='noise', zorder=1
        )

    # Plot clustered points
    non_noise_mask = ~noise_mask
    unique_labels = sorted(set(labels[non_noise_mask]))
    n_clusters = len(unique_labels)

    # Use a colormap with enough distinct colors
    cmap = plt.cm.get_cmap('tab20', max(n_clusters, 20))
    for i, cl in enumerate(unique_labels):
        mask = labels == cl
        ax.scatter(
            embeddings_2d[mask, 0],
            embeddings_2d[mask, 1],
            s=2, alpha=0.4, color=cmap(i % 20), zorder=2
        )

    ax.set_xlabel('UMAP 1')
    ax.set_ylabel('UMAP 2')
    ax.set_title(f'{title} ({n_clusters} clusters)')

    if noise_mask.any():
        n_noise = noise_mask.sum()
        ax.annotate(f'Noise: {n_noise} ({n_noise/len(labels)*100:.1f}%)',
                    xy=(0.02, 0.98), xycoords='axes fraction',
                    fontsize=9, va='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    plt.tight_layout()
    fig.savefig(FIGURES_DIR / filename, dpi=150, bbox_inches='tight')
    plt.show()
    print(f'Saved: {FIGURES_DIR / filename}')


def plot_ho_type_overlay(embeddings_2d, ho_type, df_indicators, indicator_col,
                         title_prefix, filename_prefix):
    """Scatter plot highlighting indicators of a single Ho wordplay type."""
    fig, ax = plt.subplots(figsize=(10, 8))

    # All points in gray
    ax.scatter(
        embeddings_2d[:, 0], embeddings_2d[:, 1],
        s=1, alpha=0.1, color='lightgray', zorder=1
    )

    # Highlight indicators that have this Ho label (checking the set, not primary)
    mask = df_indicators['ho_label_set'].apply(lambda s: ho_type in s).values
    n_highlighted = mask.sum()

    ax.scatter(
        embeddings_2d[mask, 0], embeddings_2d[mask, 1],
        s=4, alpha=0.5, color=HO_COLORS[ho_type], zorder=2,
        label=f'{ho_type} (n={n_highlighted:,})'
    )

    ax.set_xlabel('UMAP 1')
    ax.set_ylabel('UMAP 2')
    ax.set_title(f'{title_prefix} — Ho type: {ho_type}')
    ax.legend(loc='upper right', fontsize=9)

    filename = f'{filename_prefix}_ho_{ho_type}.png'
    plt.tight_layout()
    fig.savefig(FIGURES_DIR / filename, dpi=150, bbox_inches='tight')
    plt.show()
    print(f'Saved: {FIGURES_DIR / filename}')

### Agglomerative k=8 Visualization

In [ ]:
plot_clusters(
    embeddings_2d,
    agglo_labels_dict[8],
    title='Agglomerative Clustering (Ward, k=8)',
    filename='agglo_k8_clusters.png'
)

In [ ]:
for ho_type in HO_TYPES:
    plot_ho_type_overlay(
        embeddings_2d,
        ho_type=ho_type,
        df_indicators=df_indicators,
        indicator_col=indicator_col,
        title_prefix='Agglomerative k=8',
        filename_prefix='agglo_k8'
    )

### Best HDBSCAN Visualization

In [ ]:
plot_clusters(
    embeddings_2d,
    best_hdbscan_labels,
    title=f'HDBSCAN (eps={best_eps})',
    filename='hdbscan_best_clusters.png'
)

In [ ]:
for ho_type in HO_TYPES:
    plot_ho_type_overlay(
        embeddings_2d,
        ho_type=ho_type,
        df_indicators=df_indicators,
        indicator_col=indicator_col,
        title_prefix=f'HDBSCAN (eps={best_eps})',
        filename_prefix='hdbscan_best'
    )

---
## Section 6: Basic Qualitative Inspection

For each cluster in the agglomerative k=8 and best HDBSCAN runs, we print:
1. The 10 indicators closest to the cluster centroid (in 10D UMAP space)
2. The distribution of Ho wordplay types within that cluster

This gives a quick view of whether each cluster corresponds to a recognizable wordplay
type or is a mixture. Deeper qualitative analysis will be in Notebook 05.

In [ ]:
def inspect_clusters(labels, embeddings_10d, df_indicators, indicator_col,
                     method_name, n_examples=10):
    """Print centroid-nearest indicators and Ho type distribution per cluster."""
    unique_labels = sorted(set(labels))

    for cl in unique_labels:
        if cl == -1:
            # Summarize noise points briefly
            n_noise = (labels == -1).sum()
            print(f'\n{"=" * 60}')
            print(f'{method_name} — Noise points: {n_noise}')
            print(f'{"=" * 60}')
            continue

        mask = labels == cl
        cluster_size = mask.sum()
        cluster_embeddings = embeddings_10d[mask]
        cluster_indicators = df_indicators.loc[mask]

        # Compute centroid and find nearest points
        centroid = cluster_embeddings.mean(axis=0)
        dists_to_centroid = np.linalg.norm(cluster_embeddings - centroid, axis=1)
        nearest_idx = np.argsort(dists_to_centroid)[:n_examples]
        nearest_indicators = cluster_indicators.iloc[nearest_idx][indicator_col].tolist()

        # Ho type distribution within this cluster
        # Explode the label sets so each (indicator, type) pair is counted
        ho_exploded = cluster_indicators['ho_label_set'].explode()
        ho_exploded = ho_exploded[ho_exploded.apply(lambda x: isinstance(x, str))]

        print(f'\n{"=" * 60}')
        print(f'{method_name} — Cluster {cl} (n={cluster_size})')
        print(f'{"-" * 60}')
        print(f'Nearest to centroid: {", ".join(nearest_indicators)}')

        if len(ho_exploded) > 0:
            type_counts = ho_exploded.value_counts()
            total_labels = type_counts.sum()
            print(f'Ho type distribution ({total_labels} total label instances):')
            for wtype, count in type_counts.items():
                pct = count / total_labels * 100
                print(f'  {wtype:15s}: {count:5d} ({pct:5.1f}%)')
        else:
            print('  No Ho labels available for indicators in this cluster')

### Agglomerative k=8 — Cluster Inspection

In [ ]:
inspect_clusters(
    agglo_labels_dict[8],
    embeddings_10d,
    df_indicators,
    indicator_col,
    method_name='Agglomerative k=8'
)

### Best HDBSCAN — Cluster Inspection

HDBSCAN may produce many clusters. We inspect the 15 largest clusters to keep
output manageable. A full inspection of all clusters belongs in Notebook 05.

In [ ]:
# For HDBSCAN, inspect only the largest clusters to keep output concise
unique_hdbscan_labels = sorted(set(best_hdbscan_labels))
n_hdbscan_clusters = len([l for l in unique_hdbscan_labels if l != -1])

if n_hdbscan_clusters > 15:
    # Find the 15 largest clusters by size
    cluster_sizes = pd.Series(best_hdbscan_labels[best_hdbscan_labels != -1]).value_counts()
    top_15_labels = set(cluster_sizes.head(15).index)

    # Create a filtered label array: keep top 15, set rest to -1
    filtered_labels = np.where(
        np.isin(best_hdbscan_labels, list(top_15_labels)),
        best_hdbscan_labels,
        -1
    )
    print(f'HDBSCAN found {n_hdbscan_clusters} clusters. '
          f'Showing the 15 largest below.')
    inspect_clusters(
        filtered_labels,
        embeddings_10d,
        df_indicators,
        indicator_col,
        method_name=f'HDBSCAN (eps={best_eps})'
    )
else:
    inspect_clusters(
        best_hdbscan_labels,
        embeddings_10d,
        df_indicators,
        indicator_col,
        method_name=f'HDBSCAN (eps={best_eps})'
    )

---
## Section 7: Save All Outputs

We save cluster labels for every run so that Notebook 05 can load and evaluate them
without rerunning the clustering. Each CSV has columns `indicator` and `cluster_label`.

**Output files:**

| File | Location | Description |
|------|----------|-------------|
| `cluster_labels_hdbscan_eps_{value}.csv` | `DATA_DIR` | Labels per HDBSCAN epsilon run |
| `cluster_labels_agglo_k{value}.csv` | `DATA_DIR` | Labels per agglomerative k |
| `clustering_metrics_summary.csv` | `OUTPUT_DIR` | All metrics from all runs |
| `figures/*.png` | `OUTPUT_DIR/figures` | All visualizations |

In [ ]:
# Save HDBSCAN cluster labels for each epsilon
for eps, labels in hdbscan_labels_dict.items():
    eps_str = f'{eps:.4f}'.replace('.', 'p')  # e.g., 0.1500 -> 0p1500
    fname = f'cluster_labels_hdbscan_eps_{eps_str}.csv'
    out_df = pd.DataFrame({
        'indicator': df_indicators[indicator_col].values,
        'cluster_label': labels,
    })
    out_df.to_csv(DATA_DIR / fname, index=False)
    print(f'Saved: {fname}')

In [ ]:
# Save agglomerative cluster labels for each k
for k, labels in agglo_labels_dict.items():
    fname = f'cluster_labels_agglo_k{k}.csv'
    out_df = pd.DataFrame({
        'indicator': df_indicators[indicator_col].values,
        'cluster_label': labels,
    })
    out_df.to_csv(DATA_DIR / fname, index=False)
    print(f'Saved: {fname}')

In [ ]:
# Build a combined metrics summary for all runs
all_metrics = []

# HDBSCAN runs
for _, row in df_hdbscan.iterrows():
    all_metrics.append({
        'method': 'HDBSCAN',
        'parameters': f'min_cluster_size=10, eps={row["epsilon"]}',
        'n_clusters': int(row['n_clusters']),
        'n_noise': int(row['n_noise']),
        'noise_pct': row['noise_pct'],
        'silhouette': row['silhouette'],
        'davies_bouldin': row['davies_bouldin'],
        'calinski_harabasz': float('nan'),  # not computed for HDBSCAN (noise points)
    })

# Agglomerative runs
for _, row in df_agglo.iterrows():
    all_metrics.append({
        'method': 'Agglomerative (Ward)',
        'parameters': f'k={int(row["k"])}',
        'n_clusters': int(row['k']),
        'n_noise': 0,
        'noise_pct': 0.0,
        'silhouette': row['silhouette'],
        'davies_bouldin': row['davies_bouldin'],
        'calinski_harabasz': row['calinski_harabasz'],
    })

df_all_metrics = pd.DataFrame(all_metrics)
metrics_path = OUTPUT_DIR / 'clustering_metrics_summary.csv'
df_all_metrics.to_csv(metrics_path, index=False)

print(f'Saved: {metrics_path}')
print(f'\n--- Full Metrics Summary ---')
print(df_all_metrics.to_string(index=False))

In [ ]:
# Final file listing
print('=== All Output Files ===')
print(f'\nCluster labels (in {DATA_DIR}):')
for f in sorted(DATA_DIR.glob('cluster_labels_*.csv')):
    print(f'  {f.name}')

print(f'\nMetrics summary (in {OUTPUT_DIR}):')
print(f'  clustering_metrics_summary.csv')

print(f'\nFigures (in {FIGURES_DIR}):')
for f in sorted(FIGURES_DIR.glob('*.png')):
    print(f'  {f.name}')

print('\nDone. All outputs saved for Notebook 05.')